In [86]:
from collections import defaultdict
from dataclasses import dataclass
from datetime import timedelta
from typing import List, Union, Optional

import arrow
from arrow import Arrow
import pandas as pd

@dataclass
class Asset:
    category: str
    ticker: str
    price: float
    units: int
    date: Arrow
        
    @property
    def to_row(self):
        return [self.category, self.ticker, self.price, self.units, self.date]

@dataclass
class Event:
    action: str
    price: float
    units: int
    date: Arrow
    fmv: float
        
    @property
    def to_row(self):
        return [self.action, self.price, self.units, self.date, self.fmv]
    
class Portfolio:
    def __init__(self):
        self.assets = pd.DataFrame(columns=[
            'category', 'ticker', 'price', 'units', 'date'
        ])
        self.events = pd.DataFrame(columns=[
            'action', 'price', 'units', 'date', 'fmv'
        ])
        
    def _add_asset(self, asset: Asset):
        self.assets.loc[len(self.assets)] = asset.to_row
        self.assets.sort_values('date', inplace=True)
        
    def _add_event(self, event: Event):
        self.events.loc[len(self.events)] = event.to_row
        self.events.sort_values('date', inplace=True)
        
    def cost_basis(self, category: str, ticker: str) -> float:
        assets = self.assets.query(f"category == '{category}' and ticker == '{ticker}'")
        return (self.assets.price * self.assets.units).sum()
    
    def total_amount(self, category: str, ticker: str) -> int:
        assets = self.assets.query(f"category == '{category}' and ticker == '{ticker}'")
        return self.assets.units.sum()
    
    def grant_option(self, ticker: str, price: float, units: int, date: Arrow) -> Event:
        event = Event('option grant', price, units, date, None)
        self._add_event(event)
        option = Asset('option', ticker, price, units, date)
        self._add_asset(option)
        return event
    
    def grant_options_from_schedule(self, 
                                    ticker: str, 
                                    price: float, 
                                    units: int, 
                                    begin_date: Arrow, 
                                    cliff_date: Arrow,
                                    cutoff_date=Arrow.utcnow(),
                                    num_months=48):
        raw_chunk_size, remainder = divmod(units, num_months)
        amount_to_grant = 0
        for i in range(num_months):
            # "consume" from the remainder while it exists
            chunk_size = raw_chunk_size + 1 if i < remainder else raw_chunk_size
            amount_to_grant += chunk_size
            
            # get date at this chunk of options
            year_delta, month_delta = divmod(i, 12)
            chunk_year = begin_date.year + year_delta
            chunk_month = begin_date.month + month_delta

            # roll chunk month into year if greater than 12
            year_roll, chunk_month = divmod(chunk_month, 12)
            chunk_year += year_roll
            
            chunk_date = Arrow(chunk_year, chunk_month + 1, begin_date.day)
            
            # we can't get grants until the cliff (if it exists) is over
            beyond_cliff = cliff_date is None or chunk_date >= cliff_date
            # we want to add chunks up until this point in time unless
            # the cutoff date is manually set into the future
            not_cut_off = cutoff_date is None or chunk_date <= cutoff_date
            if beyond_cliff and not_cut_off:
                self.grant_option(ticker, price, chunk_size, chunk_date)
                chunk_size = 0
                
            
        
    def exercise_option(self, ticker: str, price: float, units: int, date: Arrow, fmv: float) -> Event:
        event = Event('option exercise', price, units, date, fmv)
        self._add_event(event)
        
        return event

p = Portfolio()
p.grant_options_from_schedule('JEFF', 2.18, 8000, Arrow(2019, 1, 7), Arrow(2020, 1, 7))
p.grant_options_from_schedule('JEFF', 3.08, 5000, Arrow(2020, 6, 1), None)
p.assets

,category,ticker,price,units,date
0,option,JEFF,2.18,167,2020-01-07T00:00:00+00:00
1,option,JEFF,2.18,167,2020-02-07T00:00:00+00:00
2,option,JEFF,2.18,167,2020-03-07T00:00:00+00:00
3,option,JEFF,2.18,167,2020-04-07T00:00:00+00:00
4,option,JEFF,2.18,167,2020-05-07T00:00:00+00:00
5,option,JEFF,2.18,167,2020-06-07T00:00:00+00:00
13,option,JEFF,3.08,105,2020-07-01T00:00:00+00:00
6,option,JEFF,2.18,167,2020-07-07T00:00:00+00:00
14,option,JEFF,3.08,105,2020-08-01T00:00:00+00:00
7,option,JEFF,2.18,167,2020-08-07T00:00:00+00:00


In [63]:
divmod(9, 12)

(0, 9)

In [60]:
begin = getattr(Arrow(2019, 1, 7), 'year')
end = getattr(Arrow(2023, 1, 7), 'year')
end - begin

4

In [25]:
filing_state = 'georgia'
filing_status = 'single'
gross_income = 127_200
withholdings = 18_000 + 2_500 + 22 # 401k, HSA, Dental
agi = gross_income - withholdings

# Leave None if taking standard deduction,
# otherwise enter value in dollars
nonstandard_state_deduction = None
nonstandard_federal_deduction = None

fmv = 22
target_price = 120
options = ((2.18, 1600), (3.08, 833))


In [2]:
import requests as rq
def get_tax_info(region: str, filing_status: str, capital_gains=False) -> (int, list):
    """Fetch tax brackets and deduction amount for a region of the US for FY2020 
    :param region: Can be any of the 50 states, `district of columbia`, or `federal`
    :param filing_status: Options are (single, married, married_separately, head_of_household)
    :param capital_gains: Whether you want to return capital gains rates instead of income
    """
    if capital_gains and region != 'federal':
        raise ValueError("Can only apply capital gains rate to `federal` region")
        
    normalized_region = region.lower().replace(' ', '_')
    url = (f"https://raw.githubusercontent.com/taxee/taxee-tax-statistics"
           f"/master/src/statistics/2020/{normalized_region}.json")
    res = rq.get(url)
    try:
        res.raise_for_status()
    except rq.RequestException as e:
        raise ValueError(f"'{region}' is not a valid region")
    if region == 'federal':
        data = res.json()['tax_withholding_percentage_method_tables']['annual'][filing_status]
    else:
        data = res.json()[filing_status]
    deduction = data['deductions'][0]['deduction_amount']
    rate_key = 'marginal_capital_gain_rate' if capital_gains else 'marginal_rate'
    brackets = data['income_tax_brackets']
    brackets = [
        {'income_level': d['bracket'], 'marginal_rate': d[rate_key] / 100.0} 
        for d in brackets
    ]
    return deduction, brackets

In [3]:
from typing import List
def taxes(income: float, brackets: List[dict]) -> float:
    """Calculate taxes owed based on progressive bracket"""
    taxes = 0
    for i, bracket in enumerate(brackets):
        if income == 0:
            break
        # if we have yet to hit the last bracket
        if i < len(brackets) - 1:
            next_bracket_income = brackets[i + 1]['income_level']
            income_level_band = next_bracket_income - bracket['income_level']
            portion = min(income, income_level_band)
        # otherwise the rest of the income will be taxed at this last bracket
        else:
            portion = income
        taxes += portion * bracket['marginal_rate']
        income -= portion

    # if there is income left over after exhausting all brackets
    # tax remainder at highest bracket
    if income:
        taxes += income * brackets[-1]['marginal_rate']
    return taxes


In [37]:
def calculate_payroll_tax(income: float) -> float:
    social_security_bracket = [{'income_level': 142_800, 'marginal_rate': 0.062}]
    medicare_rate = 0.0145
    social_security_tax = taxes(income, social_security_bracket)
    medicare_tax = gross_income * medicare_rate
    payroll_tax = social_security_tax + medicare_tax
    return payroll_tax

# payroll taxes are applied to unadjusted gross income
calculate_payroll_tax(gross_income)

9730.8

In [34]:
def calculate_income_taxes(
    income: float, 
    filing_state: str, 
    filing_status='single', 
    federal_deduction=None, 
    state_deduction=None,
) -> dict:
    standard_state_deduction, state_income_brackets = get_tax_info(filing_state, filing_status)
    standard_federal_deduction, federal_income_brackets = get_tax_info('federal', filing_status)
    
    # apply custom deductions if available
    federal_deduction = federal_deduction or standard_federal_deduction
    state_deduction = state_deduction or standard_federal_deduction
    
    federal_income_tax = taxes(income - federal_deduction, federal_income_brackets) 
    state_income_tax = taxes(income - state_deduction, state_income_brackets) 

    return {
        'federal': federal_income_tax, 
        'state': state_income_tax, 
    }
income_taxes = calculate_income_taxes(agi, filing_state)
income_taxes

{'federal': 16706.22, 'state': 5466.679999999999}

In [35]:
from typing import Tuple
def calculate_amt_taxes(income: float, options: Tuple[float, int]) -> float:
    amt_exemption = 72_900
    cost_basis = sum(o[0] * o[1] for o in options)
    number_of_options = sum(o[1] for o in options)
    avg_price_per_share = cost_basis / number_of_options
    valuation_at_exercise = number_of_options * fmv

    exercise_spread = valuation_at_exercise - cost_basis
    amt_base = agi + exercise_spread - amt_exemption
    amt_tax_rate = 0.26
    tmt = amt_base * amt_tax_rate
    amt_tax = max(0, tmt - income_taxes['federal'])
    
    return amt_tax

def calculate_capital_gains_taxes(
    income: float, 
    options: options: Tuple[float, int], 
    target_price: float, 
    long_term=False
    ) -> float:
    cost_basis = sum(o[0] * o[1] for o in options)
    number_of_options = sum(o[1] for o in options)
    
    valuation_at_sale = number_of_options * target_price
    gains_from_sale = valuation_at_sale - cost_basis
    _, cap_gains_brackets = get_tax_info('federal', 'single', capital_gains=True)
    cap_gains_tax = taxes(gains_from_sale, cap_gains_brackets)

long_strat_taxes = (amt_tax + cap_gains_tax)
short_cap_gains_taxes = calculate_income_taxes(gains_from_sale, filing_state)
short_strat_taxes = sum(short_cap_gains_taxes.values())
strat_diff = short_strat_taxes - long_strat_taxes

print(f"""
cost to exercise {number_of_options:,} shares at avg price of ${avg_price_per_share:.2f}: ${cost_basis:,.2f}
upfront amt owed if exercised at fmv of ${fmv}: ${amt_tax:,.2f}
total tax paid if held long term and sold at ${target_price}: {long_strat_taxes:,.2f}
total tax paid if sold short term: ${short_strat_taxes:,.2f}
potential savings by going long strat ${strat_diff:,.2f}
""")


cost to exercise 2,433 shares at avg price of $2.49: $6,053.64
upfront amt owed if exercised at fmv of $22: $4,418.87
total tax paid if held long term and sold at $120: 41,286.08
total tax paid if sold short term: $86,742.61
potential savings by going long strat $45,456.53

